In [1]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import keras_tuner
import tensorflow as tf
import matplotlib.pyplot as plt
import os 
import numpy as np
from tensorflow import keras
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support

/Users/rosameliacarioni/miniconda3/envs/bach_thesis_4/lib/python3.9/site-packages/tensorflow_io-0.32.0-py3.9-macosx-11.0-arm64.egg/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/Users/rosameliacarioni/miniconda3/envs/bach_thesis_4/lib/python3.9/site-packages/tensorflow_io-0.32.0-py3.9-macosx-11.0-arm64.egg/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ["[Errno 2] The file to load file system plugin from does not exist.: '/Users/rosameliacarioni/miniconda3/envs/bach_thesis_4/lib/python3.9/site-packages/tensorflow_io-0.32.0-py3.9-macosx-11.0-arm64.egg/tensorflow_io/python/ops/libtensorflow_io_plugins.so'"]
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/Users/rosameliacarioni/miniconda3/envs/bach_thesis_4/lib/python3.9/site-packages/tensorflow_io-0.32.0-py3.9-macosx-11.0-arm64.egg/tensorflow_io/python/ops/__init__.py:104: UserWarning

# After model was obtained with param optimization, we need to train it: 

## 9. Evaluate it's performance by doing k-Fold Cross Validation and improve the weights from the model 
https://machinelearningmastery.com/evaluate-performance-deep-learning-models-keras/ <br>
https://repository.tudelft.nl/islandora/object/uuid%3A6f4f3def-f8e0-4820-8b4f-75b0254dadcd <br>
https://stackoverflow.com/questions/50997928/typeerror-only-integer-scalar-arrays-can-be-converted-to-a-scalar-index-with-1d


In [ ]:
epoch = 2 #15
batch = 8
splits = 2 #10

location = 'data/models'

model = keras.models.load_model(location)
kfold = StratifiedKFold(n_splits=splits, shuffle=True, random_state=123)
acc_scores = [] # of test data 
histories = []
confusion_matrices = []
precision_test = []
recall_test = []
F1_test = []
precision_train = []
recall_train = []
F1_train = [] 

In [ ]:

for train, test in kfold.split(x, y):
    # 3. Fit the model
    x_train = np.array(x)[train.astype(int)]
    y_train = np.array(y)[train.astype(int)]
    x_test = np.array(x)[test.astype(int)]
    y_test = np.array(y)[test.astype(int)]
    
    hist = model.fit(x_train, y_train, epochs=epoch, batch_size=batch, verbose=0, validation_data = (x_test, y_test), callbacks=[stop_early])
    
    # Save information about model 
    histories.append(hist)
    
    # Display accuracy of validation set 
    # hist.history returns all the metrics. By adding: ['val_accuracy'][-1] we get only the accuracy of the testing set at the last epoch
    print("%s: %.2f%%" % (model.metrics_names[1], hist.history['val_accuracy'][-1] *100))
    acc_scores.append(hist.history['val_accuracy'][-1] * 100)

    # Store confusion matrix 
    y_pred = model.predict(x_test)
    y_pred = [1 if prediction > 0.5 else 0 for prediction in y_pred]
    confusion_mtx = tf.math.confusion_matrix(y_test, y_pred)
    confusion_matrices.append(confusion_mtx) 

    # Store precision, recall and F1 for testing=validation data 
    precision_temp, recall_temp, f1_score, _ = precision_recall_fscore_support(y_test, y_pred, average='binary')
    precision_test.append(precision_temp)
    recall_test.append(recall_temp)
    F1_test.append(f1_score)

    # Store precision, recall and F1 for train data
    y_pred = model.predict(x_train)
    y_pred = [1 if prediction > 0.5 else 0 for prediction in y_pred]

    precision_temp, recall_temp, f1_score, _ = precision_recall_fscore_support(y_train, y_pred, average='binary')
    precision_train.append(precision_temp)
    recall_train.append(recall_temp)
    F1_train.append(f1_score)

print("%.2f%% (+/- %.2f%%)" % (np.mean(acc_scores), np.std(acc_scores)))

## 10. Confusion Matrix 
https://www.tensorflow.org/tutorials/audio/simple_audio

In [ ]:
mean_matrix = np.mean(confusion_matrices, axis=0)

In [ ]:
label_names = ['Gunshot' ,'No gunshot']
plt.figure(figsize=(10, 8))
sns.heatmap(mean_matrix,
            xticklabels=label_names,
            yticklabels=label_names,
            annot=True, fmt='g')
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.title('Cunfusion matrix: original data')
plt.savefig('confusion_matrix_original_data.png')
plt.show()


## 11. Display and save graphs of other metrics

Obtain the mean of all different k-folds 

In [ ]:
list_loss = []
list_val_loss = []

list_accuracy = []
list_val_accuracy = []

for i in range(epoch):
    temp_loss = [ hist.history['loss'][i] for hist in histories ]
    list_loss.append(np.mean(temp_loss))
    temp_val_loss = [ hist.history['val_loss'][i] for hist in histories ]
    list_val_loss.append(np.mean(temp_val_loss))

    temp_accuracy = [ hist.history['accuracy'][i] for hist in histories ]
    list_accuracy.append(np.mean(temp_accuracy))
    temp_val_accuracy = [ hist.history['val_accuracy'][i] for hist in histories ]
    list_val_accuracy.append(np.mean(temp_val_accuracy))



In [ ]:
plt.title('Loss: original data')
plt.plot(list_loss, 'r')
plt.plot(list_val_loss, 'b')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['train', 'test'])
plt.grid()
#plt.savefig('loss_original_data.png')
plt.show()

In [ ]:
plt.title('Accuracy: original data')
plt.plot(list_accuracy, 'r')
plt.plot(list_val_accuracy, 'b')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['train', 'test'])
plt.grid()
#plt.savefig('accuracy_original_data.png')
plt.show()